In [1]:
import geohash as geohash
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
# https://towardsdatascience.com/simple-gps-data-visualization-using-python-and-open-street-maps-50f992e9b676
# https://github.com/tisljaricleo/GPS-visualization-Python/blob/main/gps_class.py

In [2]:
train_df = pd.read_csv('train.csv')
train_df=train_df.sample(n=500000,replace=True, random_state=1)
dup_train_df = train_df[train_df['TRIP_ID'].duplicated()]
dup_train_df.reset_index()
lst_duplicated_trip_id = list(dup_train_df['TRIP_ID']) 
cleaned_train_df = train_df[~train_df['TRIP_ID'].isin(lst_duplicated_trip_id)]

In [3]:
box=geohash.bbox("ez3fh")
box

{'s': 41.1328125, 'w': -8.61328125, 'n': 41.1767578125, 'e': -8.5693359375}

In [4]:
# Points
# points=(45.8357, 15.9645, 45.6806, 16.1557)
# points=(41.1760,-8.6130,41.1330,-8.5693)



In [5]:
def scale_to_img(points, lat_lon, h_w):
        """
        Conversion from latitude and longitude to the image pixels.
        It is used for drawing the GPS records on the map image.
        :param lat_lon: GPS record to draw (lat1, lon1).
        :param h_w: Size of the map image (w, h).
        :return: Tuple containing x and y coordinates to draw on map image.
        """
        # https://gamedev.stackexchange.com/questions/33441/how-to-convert-a-number-from-one-min-max-set-to-another-min-max-set/33445
        old = (points[2], points[0])
        new = (0, h_w[1])
        y = ((lat_lon[0] - old[0]) * (new[1] - new[0]) / (old[1] - old[0])) + new[0]
        old = (points[1], points[3])
        new = (0, h_w[0])
        x = ((lat_lon[1] - old[0]) * (new[1] - new[0]) / (old[1] - old[0])) + new[0]
        # y must be reversed because the orientation of the image in the matplotlib.
        # image - (0, 0) in upper left corner; coordinate system - (0, 0) in lower left corner
        return int(x), h_w[1] - int(y)

In [6]:
# def get_ticks(self):
#         """
#         Generates custom ticks based on the GPS coordinates of the map for the matplotlib output.
#         :return:
#         """
#         self.x_ticks = map(
#             lambda x: round(x, 4),
#             np.linspace(self.points[1], self.points[3], num=7))
#         y_ticks = map(
#             lambda x: round(x, 4),
#             np.linspace(self.points[2], self.points[0], num=8))
#         # Ticks must be reversed because the orientation of the image in the matplotlib.
#         # image - (0, 0) in upper left corner; coordinate system - (0, 0) in lower left corner
#         self.y_ticks = sorted(y_ticks, reverse=True)

In [8]:
def image_generation(image_size,original_image_name,save_image_name,points,no_of_image,foldername,box):
    count=0
    limit=0
    lat_or=[]
    long_or=[]
    ylimit_low=box['s']
    xlimit_low=box['w']
    ylimit_high=box['n']
    xlimit_high=box['e']
    for data in range(cleaned_train_df.shape[0]):
        trajectories=[]
        count=count+1
        coord = cleaned_train_df.eval(cleaned_train_df[['POLYLINE']].values[data])[0]
        if(len(coord)<1):
            print(len(coord))
        else:
            xs, ys = zip(*coord)
            if((max(xs)<=xlimit_high) and (min(xs)>= xlimit_low) and (max(ys)<= ylimit_high) and (min(ys)>= ylimit_low) ):
                if(len(xs)>40):
                    result_image = Image.open('maps/'+original_image_name, 'r')
                    img_points = []
                    for i in range(len(xs)):
                        temptuple=[]
                        temptuple.append(ys[i])
                        temptuple.append(xs[i])
                        x1,y1=scale_to_img(points,temptuple,image_size)
                        img_points.append((x1, y1))
                    draw = ImageDraw.Draw(result_image)
                    draw.line(img_points, fill=(0, 0, 0), width=3)
#                     result_image.save(foldername+"/"+str(limit)+"_"+save_image_name+".png")
                    result_image.save(foldername+"/"+str(limit)+"_"+save_image_name+".jpg")
                    limit=limit+1
                    print(limit)
                    if(limit>no_of_image):
                        print("Raw no: "+str(count))
                        break
        

In [10]:
# # Center Grid Image generation
# image_size=(128,128)
# # points=(41.1760,-8.6130,41.1330,-8.5693)
# points=(box['n'],box['w'],box['s'],box['e'])
# original_image_name="128centerimage.png"
# no_of_image=1
# foldername="CenterGrid128"
# image_generation(image_size,original_image_name,"centergrid",points,no_of_image,foldername,box)

# Center Grid Image generation
image_size=(1591,1193)
# points=(41.1760,-8.6130,41.1330,-8.5693)
points=(box['n'],box['w'],box['s'],box['e'])
original_image_name="map64.jpg"
no_of_image=30
foldername="originalimage"
image_generation(image_size,original_image_name,"centergrid",points,no_of_image,foldername,box)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
0
19
0
20
21
22
23
24
0
25
26
27
28
29
30
31
Raw no: 756


In [9]:
# Geo hash neigbours
grids=geohash.neighbors("ez3fh")

In [10]:
grids

['ez3f5', 'ez3fj', 'ez3cu', 'ez3cg', 'ez3cv', 'ez3fk', 'ez3f7', 'ez3fm']

# left

In [11]:
box=geohash.bbox(grids[0])
box

{'s': 41.1328125, 'w': -8.6572265625, 'n': 41.1767578125, 'e': -8.61328125}

In [12]:
# Center Grid Image generation
image_size=(512,512)
# points=(41.1760,-8.6130,41.1330,-8.5693)
points=(box['n'],box['w'],box['s'],box['e'])
original_image_name="left.png"
no_of_image=10
foldername="left"
image_generation(image_size,original_image_name,"left",points,no_of_image,foldername,box)

Raw no: 69


# Right

In [13]:
box=geohash.bbox(grids[1])
box

{'s': 41.1328125, 'w': -8.5693359375, 'n': 41.1767578125, 'e': -8.525390625}

In [14]:
# Center Grid Image generation
image_size=(512,512)
# points=(41.1760,-8.6130,41.1330,-8.5693)
points=(box['n'],box['w'],box['s'],box['e'])
original_image_name="right.png"
no_of_image=10
foldername="Right"
image_generation(image_size,original_image_name,"Right",points,no_of_image,foldername,box)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Raw no: 13144


# Down

In [15]:
box=geohash.bbox(grids[2])
box

{'s': 41.0888671875, 'w': -8.61328125, 'n': 41.1328125, 'e': -8.5693359375}

In [16]:
# Center Grid Image generation
image_size=(512,512)
# points=(41.1760,-8.6130,41.1330,-8.5693)
points=(box['n'],box['w'],box['s'],box['e'])
original_image_name="down.png"
no_of_image=10
foldername="down"
image_generation(image_size,original_image_name,"down",points,no_of_image,foldername,box)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Raw no: 53673


# Left Down

In [17]:
box=geohash.bbox(grids[3])
box

{'s': 41.0888671875, 'w': -8.6572265625, 'n': 41.1328125, 'e': -8.61328125}

In [18]:
# Center Grid Image generation
image_size=(512,512)
# points=(41.1760,-8.6130,41.1330,-8.5693)
points=(box['n'],box['w'],box['s'],box['e'])
original_image_name="leftdown.png"
no_of_image=10
foldername="leftdown"
image_generation(image_size,original_image_name,"leftdown",points,no_of_image,foldername,box)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Raw no: 126057


# Right Down

In [19]:
box=geohash.bbox(grids[4])
box

{'s': 41.0888671875, 'w': -8.5693359375, 'n': 41.1328125, 'e': -8.525390625}

In [20]:
# Center Grid Image generation
image_size=(512,512)
# points=(41.1760,-8.6130,41.1330,-8.5693)
points=(box['n'],box['w'],box['s'],box['e'])
original_image_name="rightdown.png"
no_of_image=10
foldername="rightdown"
image_generation(image_size,original_image_name,"rightdown",points,no_of_image,foldername,box)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


# Up

In [21]:
box=geohash.bbox(grids[5])
box

{'s': 41.1767578125, 'w': -8.61328125, 'n': 41.220703125, 'e': -8.5693359375}

In [22]:
# Center Grid Image generation
image_size=(512,512)
# points=(41.1760,-8.6130,41.1330,-8.5693)
points=(box['n'],box['w'],box['s'],box['e'])
original_image_name="up.png"
no_of_image=10
foldername="up"
image_generation(image_size,original_image_name,"up",points,no_of_image,foldername,box)

0
0
0
0
Raw no: 1245


# Up Left

In [23]:
box=geohash.bbox(grids[6])
box

{'s': 41.1767578125, 'w': -8.6572265625, 'n': 41.220703125, 'e': -8.61328125}

In [24]:
# Center Grid Image generation
image_size=(512,512)
# points=(41.1760,-8.6130,41.1330,-8.5693)
points=(box['n'],box['w'],box['s'],box['e'])
original_image_name="upleft.png"
no_of_image=10
foldername="upleft"
image_generation(image_size,original_image_name,"upleft",points,no_of_image,foldername,box)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Raw no: 11095


# Up Right

In [25]:
box=geohash.bbox(grids[7])
box

{'s': 41.1767578125, 'w': -8.5693359375, 'n': 41.220703125, 'e': -8.525390625}

In [26]:
# Center Grid Image generation
image_size=(512,512)
# points=(41.1760,-8.6130,41.1330,-8.5693)
points=(box['n'],box['w'],box['s'],box['e'])
original_image_name="upright.png"
no_of_image=1
foldername="upright"
image_generation(image_size,original_image_name,"upright",points,no_of_image,foldername,box)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Raw no: 19481
